In [170]:
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
import json
import time
import random
import string
import random
import string
from Crypto.Hash import SHA256

In [171]:
def hash_value(data):
    h = SHA256.new()
    h.update(data)
    return h.hexdigest()




def get_timestamp():
    return int(time.time())

sent_messages = []
def send_message(message):
    json_message = json.dumps(message)
    print("Sending message:", json_message)
    sent_messages.append(message)

In [172]:




def xor_bytes(bytes1, bytes2):
    return bytes([b1 ^ b2 for b1, b2 in zip(bytes1, bytes2)])


def generate_random_bytes(length):
    return bytes(random.randint(0, 255) for _ in range(length))


def generate_smart_meter_info(smart_meter_id, serial_number, gateway_key):
    x_i = generate_random_bytes(16)
    return {
        "h_sn": hash_value(serial_number.encode()),
        "n_i": hash_value(
            xor_bytes(
                (smart_meter_id + hash_value(serial_number.encode())).encode(),
                bytes.fromhex(hash_value(gateway_key + x_i)),
            )
        ),
        "x_i": x_i.hex(),
    }


smart_meter_ids = ["SM1", "SM2", "SM3"]
serial_numbers = [
    "".join(random.choice(string.digits) for _ in range(10)) for _ in range(len(smart_meter_ids))
]
gateway_key = "".join(
    random.choice(string.ascii_letters + string.digits) for _ in range(16)
).encode()

print("Serial Numbers:")
for i, serial_number in enumerate(serial_numbers):
    print(f" {i+1}. {serial_number}")

print("\nGateway Key:")
print(f" {gateway_key.decode('utf-8')}")

registration_info = {
    smart_meter_id: generate_smart_meter_info(
        smart_meter_id, serial_number, gateway_key
    )
    for smart_meter_id, serial_number in zip(smart_meter_ids, serial_numbers)
}

session_keys = {}

Serial Numbers:
 1. 7827784384
 2. 6472804363
 3. 3923838776

Gateway Key:
 hPdTnJiQmcQ0q0Nk


In [173]:
import random
import string


def generate_serial_number(length=10):
    characters = string.digits
    serial_number = "".join(random.choice(characters) for _ in range(length))
    return serial_number


def generate_smart_meter(smart_meter_id, gateway_key):
    serial_number = generate_serial_number()
    h_sn = hash_value(serial_number.encode())
    x_i = get_random_bytes(16)
    n_i = hash_value((smart_meter_id + h_sn).encode()) + hash_value(gateway_key + x_i)
    return {"h_sn": h_sn, "n_i": n_i, "x_i": x_i.hex(), "serial_number": serial_number}


def generate_smart_meters(num_smart_meters, gateway_key):
    smart_meter_ids = []
    registration_info = {}
    for i in range(num_smart_meters):
        smart_meter_id = f"SM{i+1}"
        smart_meter_ids.append(smart_meter_id)
        registration_info[smart_meter_id] = generate_smart_meter(
            smart_meter_id, gateway_key
        )
    return smart_meter_ids, registration_info


def generate_gateway_key(length=16):
    characters = string.ascii_letters + string.digits
    gateway_key = "".join(random.choice(characters) for _ in range(length)).encode()
    return gateway_key


# Example usage

# Rest of the code...

In [174]:
def smart_meter_registration(smart_meter_id, serial_number):
    h_sn = hash_value(serial_number.encode())
    message = {"smart_meter_id": smart_meter_id, "h_sn": h_sn}
    send_message(message)


In [175]:
def gateway_registration_processing(message, gateway_key):
    smart_meter_id = message["smart_meter_id"]
    h_sn = message["h_sn"]
    x_i = get_random_bytes(16)
    n_i = hash_value((smart_meter_id + h_sn).encode()) + hash_value(gateway_key + x_i)
    registration_info = {
        "smart_meter_id": smart_meter_id,
        "n_i": n_i,
        "x_i": x_i.hex(),
    }
    print("Storing registration information:", registration_info)
    response_message = {
        "smart_meter_id": smart_meter_id,
        "n_i": n_i,
        "x_i": x_i.hex(),
    }
    send_message(response_message)


def smart_meter_authentication(smart_meter_id, h_sn, n_i, x_i, gateway_key):
    timestamp = get_timestamp()
    did_i = hash_value((smart_meter_id + h_sn + x_i.hex() + str(timestamp)).encode())
    k_i = get_random_bytes(16)
    pk_i = (
        hash_value((gateway_key + x_i).hex().encode() + str(timestamp).encode())
        + k_i.hex()
    )
    b_i = hash_value((did_i + x_i.hex() + str(timestamp) + pk_i).encode())
    message = {
        "smart_meter_id": smart_meter_id,
        "did_i": did_i,
        "b_i": b_i,
        "timestamp": timestamp,
        "pk_i": pk_i,
    }
    send_message(message)

In [176]:
def gateway_authentication(message, gateway_key, registration_info):
    smart_meter_id = message["smart_meter_id"]
    did_i = message["did_i"]
    b_i = message["b_i"]
    timestamp = message["timestamp"]
    pk_i = message["pk_i"]
    h_sn = registration_info[smart_meter_id]["h_sn"]
    n_i = registration_info[smart_meter_id]["n_i"]
    x_i = bytes.fromhex(registration_info[smart_meter_id]["x_i"])
    current_timestamp = get_timestamp()
    if abs(current_timestamp - timestamp) > 300:
        print("Timestamp verification failed.")
        return
    expected_did_i = hash_value(
        (smart_meter_id + h_sn + x_i.hex() + str(timestamp)).encode()
    )
    if did_i != expected_did_i:
        print("Dynamic identity verification failed.")
        return
    expected_b_i = hash_value((did_i + x_i.hex() + str(timestamp) + pk_i).encode())
    if b_i != expected_b_i:
        print("B_i verification failed.")
        return
    k_i = bytes.fromhex(pk_i[64:])
    k_gw = get_random_bytes(16)
    pk_gw = k_gw.hex() + hash_value(k_i + str(timestamp).encode() + x_i)
    c_i = hash_value(
        did_i.encode() + str(timestamp).encode() + k_i.hex().encode() + pk_gw.encode()
    ) + hash_value(gateway_key + x_i)
    response_message = {
        "smart_meter_id": smart_meter_id,
        "c_i": c_i,
        "timestamp": timestamp,
        "pk_gw": pk_gw,
    }
    send_message(response_message)
    session_key = hash_value(k_i + k_gw)
    print("Shared session key for", smart_meter_id + ":", session_key)

In [177]:
def key_refreshment(smart_meter_id, session_key, gateway_key, x_i):
    new_session_key = hash_value(
        session_key.encode() + hash_value((gateway_key + x_i)).encode()
    )
    print("New session key for", smart_meter_id + ":", new_session_key)
    smart_meter_authentication(
        smart_meter_id,
        registration_info[smart_meter_id]["h_sn"],
        registration_info[smart_meter_id]["n_i"],
        bytes.fromhex(registration_info[smart_meter_id]["x_i"]),
        gateway_key,
    )

In [178]:
def establish_multicast_group(group_id, smart_meter_ids, gateway_key):
    group_key = get_random_bytes(16)
    for smart_meter_id in smart_meter_ids:
        x_a = get_random_bytes(16)
        session_key = bytes.fromhex(session_keys[smart_meter_id])
        cipher = AES.new(session_key, AES.MODE_EAX)
        nonce = cipher.nonce
        ciphertext, tag = cipher.encrypt_and_digest(
            json.dumps(
                {
                    "group_id": group_id,
                    "timestamp": get_timestamp(),
                    "x_a": x_a.hex(),
                    "group_key": group_key.hex(),
                }
            ).encode()
        )
        message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": (nonce + tag + ciphertext).hex(),
        }
        send_message(message)

In [179]:
def join_multicast_group(message, gateway_key):
    smart_meter_id = message["smart_meter_id"]
    timestamp = message["timestamp"]
    encrypted_data = bytes.fromhex(message["encrypted_data"])
    nonce = encrypted_data[:16]
    tag = encrypted_data[16:32]
    ciphertext = encrypted_data[32:]
    session_key = bytes.fromhex(session_keys[smart_meter_id])
    cipher = AES.new(session_key, AES.MODE_EAX, nonce=nonce)
    try:
        plaintext = cipher.decrypt_and_verify(ciphertext, tag)
        multicast_info = json.loads(plaintext.decode())
        group_id = multicast_info["group_id"]
        x_a = bytes.fromhex(multicast_info["x_a"])
        group_key = bytes.fromhex(multicast_info["group_key"])
        response_message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": ciphertext.hex(),
        }
        send_message(response_message)
        multicast_groups[group_id] = {
            "group_key": group_key,
            "members": [smart_meter_id],
        }
        print("Smart meter", smart_meter_id, "joined multicast group", group_id)
    except (ValueError, KeyError) as e:
        print(
            "Error occurred during multicast group join for smart meter", smart_meter_id
        )
        print("Error message:", str(e))

In [180]:
for i in range(len(smart_meter_ids)):
    smart_meter_registration(smart_meter_ids[i], serial_numbers[i])
for i in range(len(smart_meter_ids)):
    registration_request = {
        "smart_meter_id": smart_meter_ids[i],
        "h_sn": hash_value(serial_numbers[i].encode()),
    }
    gateway_registration_processing(registration_request, gateway_key)
for i in range(len(smart_meter_ids)):
    smart_meter_authentication(
        smart_meter_ids[i],
        registration_info[smart_meter_ids[i]]["h_sn"],
        registration_info[smart_meter_ids[i]]["n_i"],
        bytes.fromhex(registration_info[smart_meter_ids[i]]["x_i"]),
        gateway_key,
    )
    session_keys[smart_meter_ids[i]] = hash_value(
        get_random_bytes(16) + get_random_bytes(16)
    )
for i in range(len(smart_meter_ids)):
    message = {
        "smart_meter_id": smart_meter_ids[i],
        "did_i": hash_value(
            (
                smart_meter_ids[i]
                + registration_info[smart_meter_ids[i]]["h_sn"]
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
            ).encode()
        ),
        "b_i": hash_value(
            (
                hash_value(
                    (
                        smart_meter_ids[i]
                        + registration_info[smart_meter_ids[i]]["h_sn"]
                        + registration_info[smart_meter_ids[i]]["x_i"]
                        + str(get_timestamp())
                    ).encode()
                )
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
                + session_keys[smart_meter_ids[i]]
            ).encode()
        ),
        "timestamp": get_timestamp(),
        "pk_i": session_keys[smart_meter_ids[i]],
    }
    gateway_authentication(message, gateway_key, registration_info)
for i in range(len(smart_meter_ids)):
    key_refreshment(
        smart_meter_ids[i],
        session_keys[smart_meter_ids[i]],
        gateway_key,
        bytes.fromhex(registration_info[smart_meter_ids[i]]["x_i"]),
    )
multicast_groups = {}

group_id = "GROUP1"
establish_multicast_group(group_id, ["SM1", "SM2"], gateway_key)
for smart_meter_id in ["SM1", "SM2"]:
    encrypted_data = None
    for message in sent_messages:
        if message["smart_meter_id"] == smart_meter_id:
            if "encrypted_data" in message:
                encrypted_data = message["encrypted_data"]
                break
    if encrypted_data:
        message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": encrypted_data,
        }
        join_multicast_group(message, gateway_key)

Sending message: {"smart_meter_id": "SM1", "h_sn": "701f334453510d1484de77a4979edafaaaabdb2cf1f6473147ace7c1c16cc8a7"}
Sending message: {"smart_meter_id": "SM2", "h_sn": "9e7294d3218ccb1dcfb920bd4aea9ca236535e35f4417de7f0eb37a684586fba"}
Sending message: {"smart_meter_id": "SM3", "h_sn": "97856ccaa6c7504bfe9ec1d4a11ad44eb640498f3f6faee633a5525b8d49fda6"}
Storing registration information: {'smart_meter_id': 'SM1', 'n_i': '3ed384783f7b85ce2352aa9475c6d786d341b43a76e9efb94105ead5ed7d6b5740c4297567caa4aba88926c910642ea97dffe4ab2aec64aeee162ccf18b7b6bc', 'x_i': '0e050f78e4d3c07ec605efbdead8aaaf'}
Sending message: {"smart_meter_id": "SM1", "n_i": "3ed384783f7b85ce2352aa9475c6d786d341b43a76e9efb94105ead5ed7d6b5740c4297567caa4aba88926c910642ea97dffe4ab2aec64aeee162ccf18b7b6bc", "x_i": "0e050f78e4d3c07ec605efbdead8aaaf"}
Storing registration information: {'smart_meter_id': 'SM2', 'n_i': 'ca1a838f49a5628c06c5d6a43ebc202cbdae94018e84a0ea2e1ed0d6490633c12563f01f938342e53eb63b2bb3837a5e4b44393b6f87